<a href="https://colab.research.google.com/github/leehgkor/lhg/blob/main/(2)_%EA%B5%AC%EA%B0%84%EB%B3%84_%EB%B9%88%EB%8F%84_%EB%B6%84%EC%84%9D(%EA%B3%A0%EC%96%B4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# step 1: 필요한 라이브러리 임포트

!pip install koreanize-matplotlib

import os
import re
import chardet
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib


In [ ]:
# step 2: 파일 읽기 및 인코딩 확인

def read_file_with_encoding(filepath):
    with open(filepath, 'rb') as f:
        raw_data = f.read()
        detected_encoding = chardet.detect(raw_data)['encoding']
        print(f"Detected encoding: {detected_encoding}")
    with open(filepath, 'r', encoding=detected_encoding) as f:
        content = f.read()
    return content

# 파일 경로 설정
file_path = "/content/소현성록 서울대도서관본.txt"
text_data = read_file_with_encoding(file_path)
print("텍스트 (앞부분):")
print(text_data[:500])  # 첫 500자 출력

In [ ]:
# step 3: 텍스트 전처리

def remove_tags_and_content(text):
    # 태그와 태그 안의 내용을 완전히 제거
    cleaned_text = re.sub(r"<[^>]*?>.*?</[^>]*?>", "", text)  # 닫는 태그와 함께 제거
    cleaned_text = re.sub(r"<[^>]+>", "", cleaned_text)  # Self-closing 태그 제거
    return cleaned_text.strip()

cleaned_text = remove_tags_and_content(text_data)

# 전처리된 텍스트의 앞부분 출력
print("전처리된 텍스트 (앞부분):")
print(cleaned_text[:500])  # 첫 500자 출력


In [ ]:
# step 4: 텍스트 구간 분절

def split_text_into_segments(text, num_segments):
    lines = text.splitlines()
    lines = [line.strip() for line in lines if line.strip()]  # 빈 줄 제거
    segment_size = len(lines) // num_segments
    segments = [lines[i * segment_size: (i + 1) * segment_size] for i in range(num_segments)]
    segments = [" ".join(segment) for segment in segments]
    return segments

# 구간 설정
num_segments = 5  # 원하는 구간 수
segments = split_text_into_segments(cleaned_text, num_segments)
print(segments)

In [ ]:
# step 5: TF-IDF를 활용한 핵심어 추출

# 사용자 지정 불용어
custom_stopwords = ["때문", "사람", "이것", "그것", "우리", "그녀", "자기", "이다", "쇼왈", "아니니", "생각", "자네", "저것", "그때", "엇지", "니르니", "더브러", "아니", "비록", "임의", "상셰", "시니", "리오", "지라", "더라", "더니", "보뫼", "아냐", "보고", "말을" ]

vectorizer = TfidfVectorizer(max_features=500, stop_words=custom_stopwords)  # 상위 500개 단어 제한
X = vectorizer.fit_transform(segments)
feature_names = vectorizer.get_feature_names_out()

# 각 구간별로 핵심어 추출
def extract_top_keywords(tfidf_matrix, feature_names, top_n=5):
    top_keywords = {}
    for i, row in enumerate(tfidf_matrix.toarray()):
        top_indices = row.argsort()[-top_n:][::-1]  # 상위 N개 단어의 인덱스
        keywords = [(feature_names[idx], row[idx]) for idx in top_indices]
        top_keywords[f"구간 {i + 1}"] = keywords
    return top_keywords

# 핵심어 추출
top_keywords = extract_top_keywords(X, feature_names, top_n=5)


In [ ]:
# step 6: 결과 정리 및 시각화

# 데이터를 DataFrame으로 변환
data = []
for segment, keywords in top_keywords.items():
    for word, score in keywords:
        data.append({"구간": segment, "단어": word, "TF-IDF 점수": score})
keywords_df = pd.DataFrame(data)

# 피벗 테이블 생성 (단어별 구간 점수로 변환)
pivot_df = keywords_df.pivot(index="단어", columns="구간", values="TF-IDF 점수").fillna(0)

# 라인 그래프 생성
plt.figure(figsize=(12, 8))
for word in pivot_df.index:
    plt.plot(pivot_df.columns, pivot_df.loc[word], marker="o", label=word)

# 그래프 세부 설정
plt.title("구간별 핵심어 TF-IDF 점수 변화", fontsize=16)
plt.xlabel("구간", fontsize=14)
plt.ylabel("TF-IDF 점수", fontsize=14)
plt.legend(title="단어", bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=10)
plt.grid(True)
plt.tight_layout()

# 그래프 표시
plt.show()

# 결과 출력
for segment, keywords in top_keywords.items():
    print(f"\n{segment}의 상위 핵심어:")
    for word, score in keywords:
        print(f"{word}: {score:.4f}")
